In [1]:
import requests
import os
import glob
import concurrent.futures

## read txt generated by IDL

In [2]:
url_files =sorted(glob.glob('data/url*.txt'))
url_files

['data\\url_list_2010.txt',
 'data\\url_list_2011.txt',
 'data\\url_list_2012.txt',
 'data\\url_list_2013.txt',
 'data\\url_list_2014.txt']

In [3]:
urls=[]
for url_file in url_files:
    with open(url_file, 'r') as f:
        urls.extend(line.strip() for line in f.readlines())

### correct 006 to 007

In [4]:
modified_urls=[]
for url in urls:
    # only level 2 spectrum data
    if 'EVS' in url:
        # Split the URL into different parts
        base_url, filename = url.rsplit('/', 1)
        filename_parts = filename.split('_')

        # Change the desired number
        filename_parts[-2] = "007"
        filename_parts[-1] = "02.fit.gz"
        
        # Reconstruct the modified URL
        modified_url = base_url + '/' + '_'.join(filename_parts)
        modified_urls.append(modified_url)

In [10]:
modified_urls[6000:10000]

['http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2011/004/EVS_L2_2011004_23_007_02.fit.gz',
 'http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2011/005/EVS_L2_2011005_00_007_02.fit.gz',
 'http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2011/005/EVS_L2_2011005_01_007_02.fit.gz',
 'http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2011/005/EVS_L2_2011005_02_007_02.fit.gz',
 'http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2011/005/EVS_L2_2011005_03_007_02.fit.gz',
 'http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2011/005/EVS_L2_2011005_04_007_02.fit.gz',
 'http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2011/005/EVS_L2_2011005_05_007_02.fit.gz',
 'http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2011/005/EVS_L2_2011005_06_007_02.fit.gz',
 'http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2011/005/EVS_L2_2011005_07_007_02.f

## download 

In [ ]:
for url in modified_urls:
    filename = 'data/EVE/' + url.split('/')[-1]  # Extract the filename from the URL
    if not os.path.exists(filename):
        try:
            response = requests.get(url)
            response.raise_for_status()  # Check if the request was successful
            with open(filename, 'wb') as f:
                f.write(response.content)

        except requests.exceptions.RequestException as e:
            print(f"Failed to download: {url}")
            print(f"Error message: {str(e)}")

Failed to download: http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/154/EVS_L2_2010154_08_007_02.fit.gz
Error message: 404 Client Error: Not Found for url: https://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/154/EVS_L2_2010154_08_007_02.fit.gz
Failed to download: http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/155/EVS_L2_2010155_10_007_02.fit.gz
Error message: 404 Client Error: Not Found for url: https://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/155/EVS_L2_2010155_10_007_02.fit.gz
Failed to download: http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/156/EVS_L2_2010156_12_007_02.fit.gz
Error message: 404 Client Error: Not Found for url: https://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/156/EVS_L2_2010156_12_007_02.fit.gz
Failed to download: http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/167/EVS_L2_2010167_10_007_01.fit.gz
Error

## Multiple

In [11]:
def download_file(url):
    filename = 'data/EVE/' + url.split('/')[-1]  # Extract the filename from the URL

    if os.path.exists(filename):
        # print(f"Skipped download (file already exists): {filename}")
        return

    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful

        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded: {filename}")

    except requests.exceptions.RequestException as e:
        print(f"Failed to download: {url}")
        print(f"Error message: {str(e)}")


# Create a ThreadPoolExecutor with a maximum of 5 worker threads
executor = concurrent.futures.ThreadPoolExecutor(max_workers=64)

# Submit download tasks to the executor
future_to_url = {executor.submit(download_file, url): url for url in modified_urls}

# Wait for all tasks to complete
concurrent.futures.wait(future_to_url)

# Clean up the executor
executor.shutdown()

Failed to download: http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/167/EVS_L2_2010167_17_007_02.fit.gz
Error message: 404 Client Error: Not Found for url: https://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/167/EVS_L2_2010167_17_007_02.fit.gz
Failed to download: http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/168/EVS_L2_2010168_06_007_02.fit.gz
Error message: 404 Client Error: Not Found for url: https://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/168/EVS_L2_2010168_06_007_02.fit.gz
Failed to download: http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/168/EVS_L2_2010168_05_007_02.fit.gz
Error message: 404 Client Error: Not Found for url: https://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/168/EVS_L2_2010168_05_007_02.fit.gz
Failed to download: http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/168/EVS_L2_2010168_03_007_02.fit.gz
Error

In [6]:
def download_file(url):
    filename = 'data/EVE/' + url.split('/')[-1]  # Extract the filename from the URL

    if os.path.exists(filename):
        return

    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful

        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded: {filename}")

    except requests.exceptions.RequestException as e:
        print(f"Failed to download: {url}")
        print(f"Error message: {str(e)}")

    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            print(f"File not found: {url}")
        else:
            print(f"HTTP Error: {str(e)}")


# Create a ThreadPoolExecutor with a maximum of 5 worker threads
executor = concurrent.futures.ThreadPoolExecutor(max_workers=500)

# Submit download tasks to the executor
future_to_url = {executor.submit(download_file, url): url for url in modified_urls}

# Wait for all tasks to complete
concurrent.futures.wait(future_to_url)

# Clean up the executor
executor.shutdown()

Failed to download: http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/154/EVS_L2_2010154_08_007_02.fit.gz
Error message: 404 Client Error: Not Found for url: https://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/154/EVS_L2_2010154_08_007_02.fit.gz
Failed to download: http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/168/EVS_L2_2010168_04_007_02.fit.gz
Error message: 404 Client Error: Not Found for url: https://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/168/EVS_L2_2010168_04_007_02.fit.gz
Failed to download: http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/167/EVS_L2_2010167_20_007_02.fit.gz
Error message: 404 Client Error: Not Found for url: https://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/167/EVS_L2_2010167_20_007_02.fit.gz
Failed to download: http://lasp.colorado.edu/eve/data_access/evewebdataproducts/level2/2010/168/EVS_L2_2010168_14_007_02.fit.gz
Error